# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [38]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [39]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [40]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = str(cells[2]).replace("<br/>","/").replace("</td>","").replace("<td>","") # Athlete's name, splitted by "/" if it's a team
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            sex = cells[5].text.strip()  # Sex
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event,
                'sex' : sex
                })

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)

url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"
# Try with one country - FRA (France)
html = GetHTML(url)

Getdataframe(html)

Response OK, continue.


,medal,country,athlete_name,games,sport,event,sex
0,1,France,Paul MASSON,Greece,Cycling Track,sprint individual,Men's
1,1,France,Paul MASSON,Greece,Cycling Track,1/3km time trial,Men's
2,1,France,Paul MASSON,Greece,Cycling Track,10km,Men's
3,1,France,Léon FLAMENG,Greece,Cycling Track,100km,Men's
4,1,France,Eugène-Henri GRAVELOTTE,Greece,Fencing,foil individual,Men's
...,...,...,...,...,...,...,...
884,2,France,SIMON Julia/FILLON MAILLET Quentin/CHEVALIER-B...,China,Biathlon,Relay mix,Mixed
885,2,France,LEDEUX Tess,China,Freestyle Skiing,Big air,Women's
886,2,France,TRESPEUCH Chloe,China,Snowboard,Snowboard Cross,Women's
887,3,France,FAIVRE Mathieu,China,Alpine Skiing,giant slalom,Men's


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [41]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [42]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [43]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event,sex
397,2,France,Daniel ROBIN,Mexico,Wrestling Greco-Roman,70 - 78kg (welterweight),Men's
339,3,France,Alfred TONELLO/Jacques ANQUETIL/Claude ROUER,Finland,Cycling Road,team time trial,Men's
348,2,France,Marcel RENAUD/Georges DRANSART,Australia / Sweden,Canoe Sprint,C-2 10000m,Men's
788,1,France,France,Brazil,Equestrian Eventing,team,Mixed
567,3,France,Жирар-Лено,United States,Athletics,100m hurdles,Women's


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [44]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [45]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [46]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 and len(data[0]) == 3:
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [47]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
143,TJK,Tajikistan,Asia,1996,2021
152,TUV,Tuvalu,Oceania,2008,2021
76,KAZ,Kazakhstan,Asia,1994,2022
103,MGL,Mongolia,Asia,1964,2022
15,BLR,Belarus,Europe,1994,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [48]:
i = 0
if i == 1:
    # Run scraping on all countries 
    code_countries = df_countries["Code"]
    name_countries = df_countries["Country"]
    i = 0

    for country, country_name in zip(code_countries, name_countries) :
        url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
        html = GetHTML(url)
        df_temp = Getdataframe(html)
        if i == 0 :
            df_olympic = df_temp
            i += 1
        else : 
            df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

        
        print("for : {}".format(country_name))

In [49]:
#df_olympic.to_csv("DB_olympic.csv", sep=";", index=False)

In [50]:
df_olympic = pd.read_csv("DB_olympic.csv", delimiter=";")
df_olympic.fillna("", inplace=True)

In [51]:
df_olympic.describe(include="object")

,country,athlete_name,games,sport,event,sex
count,12317,12317,12317,12317,12317,12317
unique,119,9998,25,76,659,3
top,Germany,,United States,Athletics,individual,Men's
freq,922,309,1528,1464,357,8052


In [52]:
from unidecode import unidecode

df_olympic.athlete_name = df_olympic.athlete_name.apply(lambda s: unidecode(s))

In [53]:
df_olympic.sample(10)

,medal,country,athlete_name,games,sport,event,sex
3992,1,Finland,Kalevi HAMALAINEN,United States,Cross Country Skiing,50km,Men's
7783,3,Japan,Shoryu NAN,Germany,Athletics,marathon,Men's
3765,2,Finland,Heikki KAHKONEN,Belgium,Wrestling Greco-Roman,- 60kg (featherweight),Men's
7047,2,Italy,Vincenzo PINTON,Great Britain,Fencing,sabre individual,Men's
1093,2,Bahrain,KIRWA Eunice Jepkirui,Brazil,Athletics,marathon,Women's
6671,3,Hungary,MARTON Anita,Brazil,Athletics,shot put,Women's
4704,2,France,Jeannie LONGO-CIPRELLI,Spain,Cycling Road,individual road race,Women's
1337,2,Belgium,Filip Meirkhege,Australia,Cycling Mountain Bike,cross-country,Men's
11440,2,Sweden,Johan OLSSON,Russian Federation,Cross Country Skiing,15km,Men's
4969,2,France,Arnaud BOVOLENTA,Russian Federation,Freestyle Skiing,Ski Cross,Men's


### Transform

In [54]:
i = 0
if i == 1:
    # Check chars non-ASCII return True
    def contains_non_ascii(strings):
        """
        Return: Boolean
        """
        return any(ord(char) > 127 for char in strings)


    # Filter non-ASCII & ASCII name
    df_non_ascii = df_olympic[df_olympic['athlete_name'].apply(contains_non_ascii)]
    df_ascii = df_olympic[~df_olympic['athlete_name'].apply(contains_non_ascii)]

In [56]:
i = 0
if i == 1:
    df_non_ascii[df_non_ascii["athlete_name"]=="Oscar VARONA/Ruperto HERRERA TABIO/Franklin STANDARD/Tomas HERRERA MARTINEZ/Pedro CHAPPE GARCIA/Rafael CANIZARES POEY/Juan DOMECQ/Conrado PEREZ ARMENTEROS/Miguel CALDERON GOMEZ/Juan ROCA/Alejandro URGELLÉS/Jose Miguel ALVAREZ POZO"]["athlete_name"].index

In [57]:
i = 0
if i == 1:
    strings = df_non_ascii[df_non_ascii["athlete_name"]=="Oscar VARONA/Ruperto HERRERA TABIO/Franklin STANDARD/Tomas HERRERA MARTINEZ/Pedro CHAPPE GARCIA/Rafael CANIZARES POEY/Juan DOMECQ/Conrado PEREZ ARMENTEROS/Miguel CALDERON GOMEZ/Juan ROCA/Alejandro URGELLÉS/Jose Miguel ALVAREZ POZO"]["athlete_name"].item()

    for i,char in enumerate(strings):
        if ord(char) > 127 :
            print(strings[i])

In [58]:
i = 0
if i == 1:
    # Translate all Non-ASCII name in ASCII (latine) name
    from transliterate import translit

    df_non_ascii.athlete_name = df_non_ascii.athlete_name.apply(lambda s : translit(s, reversed=True))

In [59]:
# Regroup both Dataframe, now all in Latine
#df_final = pd.concat([df_ascii, df_non_ascii])

In [60]:
#df_final.sample(10)

In [61]:
# Check if there are more than 1 athlete
def is_team(strings):
    """
    Return: result in Boolean
    """
    return len(strings.split("/")) > 1

# Team athletes
df_final_team = df_olympic[df_olympic["athlete_name"].apply(is_team)]

# Solo athlete
df_final_solo = df_olympic[~df_olympic["athlete_name"].apply(is_team)]

To do :
- Get simple sport list
- Get simple event list (weight, lenght, heigh, etc. split ?)
- Look for empty name (sometimes filled with country name)